# SPLADE for Portuguese

Inspired by https://github.com/naver/splade

In [ ]:
!nvidia-smi

Wed Jun 14 15:41:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
user = "leonardo" # monique
if user == "monique":
    main_dir = '/content/gdrive/MyDrive/Unicamp-projeto-final/'
else:
    main_dir = '/content/gdrive/MyDrive/Unicamp/IA368-DD/'

## Libraries installation

In [ ]:
%%shell
pip install pytrec_eval
pip install git+https://github.com/naver/splade.git -q
pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293465 sha256=53390d01d0f069cc59990544a3e5fa3d3c11d4b7eb732503530254b04bafa59e
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 67.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pytrec_eval
pytrec_eval.supported_measures

{'11pt_avg',
 'G',
 'P',
 'Rndcg',
 'Rprec',
 'Rprec_mult',
 'binG',
 'bpref',
 'gm_bpref',
 'gm_map',
 'infAP',
 'iprec_at_recall',
 'map',
 'map_cut',
 'ndcg',
 'ndcg_cut',
 'ndcg_rel',
 'num_nonrel_judged_ret',
 'num_q',
 'num_rel',
 'num_rel_ret',
 'num_ret',
 'recall',
 'recip_rank',
 'relative_P',
 'relstring',
 'runid',
 'set_F',
 'set_P',
 'set_map',
 'set_recall',
 'set_relative_P',
 'success',
 'utility'}

## Copying data files to the expected data structure

In [ ]:
!git clone https://github.com/naver/splade.git

Cloning into 'splade'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 667 (delta 109), reused 117 (delta 71), pack-reused 467
Receiving objects: 100% (667/667), 3.13 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (351/351), done.


In [ ]:
import shutil
import os

shutil.copyfile(f"{main_dir}Projeto Final/data.zip", "/content/data.zip")
!unzip data.zip

Archive:  data.zip
   creating: content/data/
   creating: content/data/m_marco/
  inflating: content/data/m_marco/queries_train.tsv  
   creating: content/data/m_marco/.ipynb_checkpoints/
  inflating: content/data/m_marco/triplets.tsv  
  inflating: content/data/m_marco/queries_dev.full.tsv  
  inflating: content/data/m_marco/queries_dev.tsv  
  inflating: content/data/m_marco/corpus.tsv  
   creating: content/data/m_robust/
  inflating: content/data/m_robust/qrels.robust04.txt  
   creating: content/data/m_robust/.ipynb_checkpoints/
  inflating: content/data/m_robust/queries.tsv  
  inflating: content/data/m_robust/corpus.tsv  


In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/config_splade_pt.yaml" splade/conf/

In [ ]:
#!mkdir -p splade/data/pt/triplets
#!cp content/data/m_marco/triplets.tsv splade/data/pt/triplets/raw.tsv

# LEONARDO: ALTEREI AQUI PRA PEGAR UM ARQUIVO MENOR DE TRIPLETS
!mkdir -p splade/data/pt/triplets
!head -n 10000000 content/data/m_marco/triplets.tsv > splade/data/pt/triplets/raw.tsv

In [ ]:
!wc -l splade/data/pt/triplets/raw.tsv

10000000 splade/data/pt/triplets/raw.tsv


In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/data/pt.yaml" splade/conf/train/data

### Generating/copying validation collection

Now we need to generate a validation collection: it will be based on the same validation data used in https://github.com/naver/splade.

In [ ]:
import pandas as pd

In [ ]:
!mkdir -p "content/data/m_marco/val_retrieval/collection"

In [ ]:
shutil.copyfile(f"{main_dir}Projeto Final/data_m_marco_val_retrieval.zip", "/content/data_m_marco_val_retrieval.zip")
!unzip data_m_marco_val_retrieval.zip

Archive:  data_m_marco_val_retrieval.zip
   creating: content/data/m_marco/val_retrieval/queries/
  inflating: content/data/m_marco/val_retrieval/queries/raw.tsv  
  inflating: content/data/m_marco/val_retrieval/collection/raw.tsv  


In [ ]:
import csv
import tqdm

#Lookups the respective translated passages.
def gen_val_collection():
  df_val_collection = pd.read_csv(f"{main_dir}Projeto Final/msmarco/val_retrieval/collection/raw.tsv", sep='\t', header=None)
  df_corpus = pd.read_csv("content/data/m_marco/corpus.tsv", sep='\t', header=None)

  with open("content/data/m_marco/val_retrieval/collection/raw.tsv", 'w') as f:
    writer = csv.writer(f, delimiter='\t')  # Set the delimiter as a tab

    for index, row in tqdm.tqdm(df_val_collection.iterrows(), total=df_val_collection.shape[0]):
      doc_id = row[0]
      doc_df = df_corpus[df_corpus[0] == doc_id]
      if doc_df.shape[0] >= 1:
        doc_text = doc_df.iloc[0,1]
        writer.writerow([doc_id, doc_text])

In [ ]:
!mkdir -p splade/data/pt/val_retrieval/collection
!cp content/data/m_marco/val_retrieval/collection/raw.tsv splade/data/pt/val_retrieval/collection

### Generating/copying validation queries

In [ ]:
!mkdir -p "content/data/m_marco/val_retrieval/queries"

In [ ]:
import csv
import tqdm

def gen_val_queries():
  df_val_queries = pd.read_csv(f"{main_dir}Projeto Final/msmarco/val_retrieval/queries/raw.tsv", sep='\t', header=None)
  df_queries_dev_full = pd.read_csv("content/data/m_marco/queries_dev.full.tsv", sep='\t', header=None)

  with open("content/data/m_marco/val_retrieval/queries/raw.tsv", 'w') as f:
    writer = csv.writer(f, delimiter='\t')  # Set the delimiter as a tab

    for index, row in tqdm.tqdm(df_val_queries.iterrows(), total=df_val_queries.shape[0]):
      q_id = row[0]
      q_df = df_queries_dev_full[df_queries_dev_full[0] == q_id]
      if q_df.shape[0] >= 1:
        q_text = q_df.iloc[0,1]
        writer.writerow([q_id, q_text])

In [ ]:
!mkdir -p splade/data/pt/val_retrieval/queries
!cp content/data/m_marco/val_retrieval/queries/raw.tsv splade/data/pt/val_retrieval/queries

### Copying QREL validation data

In [ ]:
!cp "{main_dir}Projeto Final/msmarco/val_retrieval/qrel.json" splade/data/pt/val_retrieval

### Copying full collection

For indexing we'll validate the model on mROBUST dataset.  So, its corpus should be used as full collection.

In [ ]:
!mkdir -p splade/data/pt/full_collection

In [ ]:
!cp content/data/m_robust/corpus.tsv splade/data/pt/full_collection/raw.tsv

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/index/pt.yaml" splade/conf/index

### Retrieval and evaluation configuration

Here, we'll configure retrieval and evaluation for mROBUST dataset.

In [ ]:
!mkdir -p splade/data/pt/dev_queries

In [ ]:
!cp content/data/m_robust/queries.tsv splade/data/pt/dev_queries/raw.tsv

Converts QREL information to JSON fomat.

In [ ]:
#TODO: Devo fazer o dump apenas dos que têm relevância 1?
from collections import defaultdict
import json

qrel = defaultdict(dict)

with open("content/data/m_robust/qrels.robust04.txt", 'r') as file:
  for line in file:
    fields = line.split()
    q_id = fields[0]
    doc_id = fields[2]
    rel = fields[3]
    qrel[q_id][doc_id] = rel

with open('splade/data/pt/dev_qrel.json', 'w') as file:
    json.dump(qrel, file)

In [ ]:
#TODO: Checar o threshold a ser usado

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/retrieve_evaluate/pt.yaml" splade/conf/retrieve_evaluate

### Flops estimation

We'll use the same test set (mRobust) to estimate flops.

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/flops/pt.yaml" splade/conf/flops

## Training, indexing and retrieving

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/model/splade_bertimbau_base.yaml" splade/conf/train/model/

In [ ]:
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/config_splade_pt.yaml" splade/conf/
!cp "{main_dir}Projeto Final/Experimentos/splade/conf/train/config/splade_pt.yaml" splade/conf/train/config

In [ ]:
%%shell
cd splade/
export PYTHONPATH=$PYTHONPATH:$(pwd)
export SPLADE_CONFIG_NAME="config_splade_pt.yaml"
python3 -m splade.train \
  config.checkpoint_dir=experiments/pt/checkpoint \
  config.index_dir=experiments/pt/index \
  config.out_dir=experiments/pt/out

Streaming output truncated to the last 5000 lines.
 35% 6090/17269 [05:19<09:28, 19.67it/s]
 35% 6093/17269 [05:19<09:27, 19.68it/s]
 35% 6095/17269 [05:19<09:39, 19.29it/s]
 35% 6098/17269 [05:20<09:14, 20.14it/s]
 35% 6101/17269 [05:20<09:19, 19.97it/s]
 35% 6103/17269 [05:20<09:25, 19.75it/s]
 35% 6105/17269 [05:20<09:46, 19.05it/s]
 35% 6107/17269 [05:20<10:12, 18.21it/s]
 35% 6109/17269 [05:20<10:13, 18.21it/s]
 35% 6111/17269 [05:20<10:12, 18.21it/s]
 35% 6113/17269 [05:20<10:05, 18.41it/s]
 35% 6115/17269 [05:21<10:10, 18.26it/s]
 35% 6118/17269 [05:21<09:27, 19.63it/s]
 35% 6121/17269 [05:21<09:36, 19.33it/s]
 35% 6123/17269 [05:21<09:48, 18.95it/s]
 35% 6125/17269 [05:21<09:44, 19.07it/s]
 35% 6127/17269 [05:21<09:36, 19.32it/s]
 35% 6130/17269 [05:21<09:12, 20.17it/s]
 36% 6133/17269 [05:21<09:39, 19.22it/s]
 36% 6136/17269 [05:22<09:26, 19.64it/s]
 36% 6139/17269 [05:22<09:17, 19.96it/s]
 36% 6141/17269 [05:22<09:38, 19.25it/s]
 36% 6143/17269 [05:22<09:40, 19.18it/s]
 36% 6

In [ ]:
!zip -r experiments.zip /content/splade/experiments

  adding: content/splade/experiments/ (stored 0%)
  adding: content/splade/experiments/pt/ (stored 0%)
  adding: content/splade/experiments/pt/checkpoint/ (stored 0%)
  adding: content/splade/experiments/pt/checkpoint/training_perf.txt (deflated 78%)
  adding: content/splade/experiments/pt/checkpoint/model/ (stored 0%)
  adding: content/splade/experiments/pt/checkpoint/model/special_tokens_map.json (deflated 40%)
  adding: content/splade/experiments/pt/checkpoint/model/vocab.txt (deflated 52%)
  adding: content/splade/experiments/pt/checkpoint/model/model.tar (deflated 10%)
  adding: content/splade/experiments/pt/checkpoint/model/pytorch_model.bin (deflated 7%)
  adding: content/splade/experiments/pt/checkpoint/model/tokenizer.json (deflated 72%)
  adding: content/splade/experiments/pt/checkpoint/model/config.json (deflated 53%)
  adding: content/splade/experiments/pt/checkpoint/model/tokenizer_config.json (deflated 35%)
  adding: content/splade/experiments/pt/checkpoint/val_full_ranki

In [ ]:
shutil.copyfile("/content/experiments.zip", f"{main_dir}Projeto Final/experiments.zip",)

'/content/gdrive/MyDrive/Unicamp/IA368-DD/Projeto Final/experiments.zip'